# Customer Support Agent with Reflection Pattern
## Building a Self-Improving Customer Support Agent
In this notebook, we'll create a customer support agent that can:
- Handle customer inquiries
- Reflect on its responses
- Improve its communication
- Track customer satisfaction
- Learn from past interactions

### 1. Initial Setup

In [1]:
import os
from openai import OpenAI
import json
from datetime import datetime

mallam_api_key = os.getenv("MALLAM_API_KEY")

client = OpenAI(
    base_url="https://api.mesolitica.com",
    api_key=mallam_api_key
)

### 2. Create Support Agent Functions

In [3]:
def analyze_sentiment(message):
    """Analyze the customer's message for sentiment and urgency"""
    sentiment_prompt = [
        {
            "role": "system",
            "content": """Analyze the customer message and return a JSON with:
            - sentiment: (positive, neutral, negative)
            - urgency: (low, medium, high)
            - emotion: (main emotion detected)
            """
        },
        {
            "role": "user",
            "content": message
        }
    ]
    
    response = client.chat.completions.create(
        model="mallam-small",
        messages=sentiment_prompt
    )
    
    return json.loads(response.choices[0].message.content)

In [4]:
def support_response(message, sentiment_analysis):
    """Generate an appropriate support response based on the message and sentiment"""
    support_prompt = [
        {
            "role": "system",
            "content": f"""You are a helpful customer support agent.
            Customer sentiment: {sentiment_analysis['sentiment']}
            Customer emotion: {sentiment_analysis['emotion']}
            Urgency level: {sentiment_analysis['urgency']}
            
            Adjust your tone and response accordingly.
            If the customer is upset, be extra empathetic.
            If urgent, provide immediate next steps.
            Always be professional and helpful."""
        },
        {
            "role": "user",
            "content": message
        }
    ]
    
    response = client.chat.completions.create(
        model="mallam-small",
        messages=support_prompt
    )
    
    return response.choices[0].message.content

In [5]:
def reflect_on_support_response(original_message, response, sentiment):
    """Reflect on the support response to ensure quality and empathy"""
    reflection_prompt = [
        {
            "role": "system",
            "content": """Analyze this customer support interaction and provide a JSON with:
            - tone_appropriate: (yes/no)
            - addressed_issue: (yes/no)
            - empathy_level: (1-5)
            - suggestions: [list of improvements]
            - strong_points: [what was done well]
            """
        },
        {
            "role": "system",
            "content": f"""Customer message: {original_message}
            Customer sentiment: {sentiment}
            Agent response: {response}"""
        }
    ]
    
    reflection = client.chat.completions.create(
        model="mallam-small",
        messages=reflection_prompt,
        max_tokens=1024
    )
    
    return json.loads(reflection.choices[0].message.content)

In [6]:
def improve_support_response(original_message, original_response, reflection):
    """Generate an improved response based on reflection feedback"""
    if reflection['tone_appropriate'] == 'yes' and reflection['addressed_issue'] == 'yes' and reflection['empathy_level'] >= 4:
        return original_response
    
    improvement_prompt = [
        {
            "role": "system",
            "content": f"""Improve this customer support response.
            Focus on these areas: {', '.join(reflection['suggestions'])}
            Keep these strong points: {', '.join(reflection['strong_points'])}"""
        },
        {
            "role": "user",
            "content": f"""Original messages: {original_message}
            Original response: {original_response}"""
        }
    ]
    
    improved_response = client.chat.completions.create(
        model="mallam-small",
        messages=improvement_prompt,
        max_tokens=512
    )
    
    return improved_response.choices[0].message.content

### 3. Create Main Support Handler

In [7]:
def handle_support_request(customer_message):
    """Handle a customer support request with refleection and improvement"""
    # track timing
    start_time = datetime.now()
    
    # 1. Analyze sentiment
    sentiment = analyze_sentiment(customer_message)
    
    # 2. Generate initial response
    initial_response = support_response(customer_message, sentiment)
    
    # 3. Reflection
    reflection = reflect_on_support_response(
        customer_message,
        initial_response,
        sentiment
    )
    
    # 4. Improve response if needed
    final_response = improve_support_response(
        customer_message,
        initial_response,
        reflection
    )
    
    # calaculate response time
    response_time = (datetime.now() - start_time).total_seconds()
    
    return {
        "sentiment_analysis": sentiment,
        "initial_response": initial_response,
        "reflection": reflection,
        "final_response": final_response,
        "response_time": response_time
    }

### 4. Test the Support Agent

In [8]:
# Test case 1: Frustrated customer with billing issue
print("Test case 1: Frustrated Customer")
result = handle_support_request(
    "I've been charged twice this month and nobody is helping me! This is ridiculous!"
)
print("\nSentiment:", result["sentiment_analysis"])
print("\nInitial Response:", result["initial_response"])
print("\nReflection:", result["reflection"])
print("\nFinal Response", result["final_response"])
print("\nResponse Time:", result["response_time"])

Test case 1: Frustrated Customer

Sentiment: {'sentiment': 'negative', 'urgency': 'high', 'emotion': 'anger'}

Initial Response: I'm sorry to hear about your issue with duplicate charges. I understand how frustrating it can be to deal with billing errors, especially when it affects your financial statements. Let's work together to resolve this as quickly as possible.

First, could you please provide me with your account number or any other identification details so that I can look up your transaction history? Once I have that information, I'll be able to confirm whether there were indeed two charges made to your account this month.

Additionally, it would be helpful if you could specify the dates of these charges and any reference numbers or descriptions associated with them. This information will allow me to trace the transactions more accurately.

Once I've gathered the necessary details, I'll investigate the charges on your behalf. My goal is to ensure that everything is in order an

In [9]:
# Test case 2: Technical support request
print("Test case 2: Technical support")
result = handle_support_request(
    "Hi, I'm having trouble logging into my account. Can you help?"
)
print("\nSentiment:", result["sentiment_analysis"])
print("\nInitial Response:", result["initial_response"])
print("\nReflection:", result["reflection"])
print("\nFinal Response", result["final_response"])
print("\nResponse Time:", result["response_time"])

Test case 2: Technical support

Sentiment: {'sentiment': 'neutral', 'urgency': 'medium', 'emotion': 'frustration'}

Initial Response: Of course, I'd be happy to assist you with that. It sounds like you're experiencing some issues logging into your account. Could you please tell me what exactly is happening when you try to log in? Are you getting any error messages or is the login just not responding?

Reflection: {'tone_appropriate': 'yes', 'addressed_issue': 'yes', 'empathy_level': 4, 'suggestions': [], 'strong_points': ['The agent is approachable and offers help.', 'They ask for more details to understand the problem better.', 'Agent shows empathy by acknowledging frustration.']}

Final Response Of course, I'd be happy to assist you with that. It sounds like you're experiencing some issues logging into your account. Could you please tell me what exactly is happening when you try to log in? Are you getting any error messages or is the login just not responding?

Response Time: 6.17648

### 5. Track Support Metrics

In [10]:
def track_support_interaction(interaction_data):
    """
    Track metrics for this support interaction
    """
    metrics = {
        "timestamp": datetime.now().isoformat(),
        "response_time": interaction_data["response_time"],
        "sentiment": interaction_data["sentiment_analysis"]["sentiment"],
        "urgency": interaction_data["sentiment_analysis"]["urgency"],
        "empathy_level": interaction_data["reflection"]["empathy_level"],
        "needed_improvement": interaction_data["initial_response"] != interaction_data["final_response"],
        "improvement_areas": interaction_data["reflection"]["suggestions"]
    }
    
    return metrics

# Example of tracking metrics
result = handle_support_request(
    "I've been a customer for 3 years and love your service, but I found a small bug in the latest update."
)
metrics = track_support_interaction(result)
print("\nSupport Interaction Metrics:")
print(json.dumps(metrics, indent=2))


Support Interaction Metrics:
{
  "timestamp": "2025-01-09T02:21:01.220185",
  "response_time": 13.489313,
  "sentiment": "positive",
  "urgency": "low",
  "empathy_level": 5,
  "needed_improvement": false,
  "improvement_areas": []
}
